# 앙상블 학습

In [12]:
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()
import warnings
warnings.filterwarnings('ignore')

In [4]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
cancer_scaled = scaler.fit_transform(cancer.data)

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    cancer.data, cancer.target, stratify=cancer.target,
    test_size=0.2, random_state=2021
)

#### 앙상블 학습을 위한 분류기
- 로지스틱 회귀
- 서포트 벡터머신
- K 최근접 이웃

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [7]:
lrc = LogisticRegression()
svc = SVC()
knn = KNeighborsClassifier()

In [9]:
from sklearn.ensemble import VotingClassifier

voc = VotingClassifier(
    estimators=[('LR',lrc), ("SVC",svc),('KMN',knn)], voting='hard'
)

In [13]:
voc.fit(X_train, y_train)
voc.score(X_test, y_test)

0.9473684210526315

- 개별학습의 성능

In [14]:
lrc.fit(X_train, y_train)
svc.fit(X_train, y_train)
knn.fit(X_train, y_train)
lrc.score(X_test, y_test), svc.score(X_test, y_test), knn.score(X_test, y_test)

(0.9649122807017544, 0.9385964912280702, 0.9473684210526315)

- 소프트 보팅

In [15]:
print(dir(lrc))

['C', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_check_n_features', '_estimator_type', '_get_param_names', '_get_tags', '_more_tags', '_predict_proba_lr', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_validate_data', 'class_weight', 'classes_', 'coef_', 'decision_function', 'densify', 'dual', 'fit', 'fit_intercept', 'get_params', 'intercept_', 'intercept_scaling', 'l1_ratio', 'max_iter', 'multi_class', 'n_features_in_', 'n_iter_', 'n_jobs', 'penalty', 'predict', 'predict_log_proba', 'predict_proba', 'random_state', 'score', 'set_params', 'solver', 'sparsify', 'tol', 'verbose', 'warm_start']


In [16]:
lrc.predict_proba(X_test[:5])

array([[3.45344934e-01, 6.54655066e-01],
       [9.99996354e-01, 3.64599332e-06],
       [7.87113497e-02, 9.21288650e-01],
       [8.36809861e-03, 9.91631901e-01],
       [7.24336946e-03, 9.92756631e-01]])

In [17]:
svc.predict_proba(X_test[:5])

AttributeError: predict_proba is not available when  probability=False

In [18]:
svc2 = SVC(probability=True)
svc2.fit(X_train, y_train)
svc2.predict_proba(X_test[:5])

array([[8.64286308e-01, 1.35713692e-01],
       [9.99352000e-01, 6.47999798e-04],
       [4.90625320e-01, 5.09374680e-01],
       [3.75268576e-02, 9.62473142e-01],
       [3.64002546e-02, 9.63599745e-01]])

In [19]:
voc2 = VotingClassifier(
    estimators=[('LR',lrc), ('SVC',svc2),("KNN",knn)], voting='soft'
)
voc2.fit(X_train,y_train)
voc2.score(X_test,y_test)

0.956140350877193

In [20]:
voc2.predict_proba(X_test[:5])

array([[6.04773825e-01, 3.95226175e-01],
       [9.99809940e-01, 1.90059852e-04],
       [2.57138877e-01, 7.42861123e-01],
       [1.47516878e-02, 9.85248312e-01],
       [1.40109922e-02, 9.85989008e-01]])

# Random Forest

In [21]:
from sklearn.ensemble import RandomForestClassifier

In [22]:
rfc = RandomForestClassifier(random_state=2021)
rfc.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 2021,
 'verbose': 0,
 'warm_start': False}

In [23]:
rfc.fit(X_train, y_train)
rfc.score(X_test, y_test)

0.9736842105263158

# XGBoost